Balloon Flight Prediction Tool



In [1]:
import sys
import os
import pathlib
import xml.dom.minidom as md
from pathlib import Path
import kml2geojson as kg
import folium
import json
import geojson
from folium import plugins
from folium.plugins import HeatMap
import numpy as np
from datetime import datetime, timedelta, time
from astra.simulator import *
import statistics

C:\Users\bcarroll\AppData\Local\Continuum\anaconda3\lib\site-packages\deap-1.2.2-py3.6.egg\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.


In [2]:
np.random.seed(62)

In [3]:
launch_datetime = datetime(2018, 11, 26, 10,0,0,0)
simEnvironment = forecastEnvironment(launchSiteLat=35.39730,      # deg
                                         launchSiteLon=-90.9965,     # deg
                                         launchSiteElev=100,           # m
                                         dateAndTime=launch_datetime,
                                         forceNonHD=True,
                                         debugging=True)

In [4]:
simFlight = flight(environment=simEnvironment,
                       balloonGasType='Helium',
                       balloonModel='HW1600',
                       nozzleLift=7.2,                                # kg
                       payloadTrainWeight=5.4,                    # kg
                       parachuteModel='SPH54',
                       numberOfSimRuns=500,
                       trainEquivSphereDiam=0.3,                    # m
                       floatingFlight=False,
                       excessPressureCoeff=1,
                       #outputFile=os.path.join('.', 'astra_output'),
                       outputFile= os.path.join('data', 'balloon_track' + '.kml'),
                       debugging=False,
                       log_to_file=True)

2018-11-21 16:36:15,271.271 DEBUG:astra.weather Fetched time zone data about the launch site: UTC offset is 0.000000 hours
2018-11-21 16:36:15,272.272 DEBUG:astra.weather Using UTC time 26/11/18 10:00
2018-11-21 16:36:15,274.274 INFO:astra.weather Preparing to download weather data for parameters:
2018-11-21 16:36:15,276.276 DEBUG:astra.weather     Launch site Latitude: 35.3973
2018-11-21 16:36:15,277.277 DEBUG:astra.weather     Launch site Longitude: -90.9965
2018-11-21 16:36:15,279.279 DEBUG:astra.weather     Launch time: 2018-11-26 10:00:00
2018-11-21 16:36:15,281.281 DEBUG:astra.GFS Attempting to download cycle data.
2018-11-21 16:36:15,283.283 DEBUG:astra.GFS Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20181121/gfs_0p50_12z.ascii?tmpprs[39:40][0:46][244:256][526:550]


2018-11-21 16:36:18,991.991 DEBUG:astra.GFS Requesting weather urls asynchronously: status will be sent to requests logger
2018-11-21 16:36:22,971.971 DEBUG:astra.GFS Updating Download progress: 25.0%% complete


2018-11-21 16:36:25,229.229 DEBUG:astra.GFS Updating Download progress: 50.0%% complete


2018-11-21 16:36:25,384.384 DEBUG:astra.GFS Updating Download progress: 75.0%% complete


2018-11-21 16:36:25,539.539 DEBUG:astra.GFS Updating Download progress: 100.0%% complete


Weather downloaded.


2018-11-21 16:36:26,226.226 DEBUG:astra.GFS Forecast successfully downloaded!
2018-11-21 16:36:26,227.227 DEBUG:astra.weather GFS data successfully downloaded.


In [5]:
simFlight.run()
#simFlight.write_CSV(os.path.join('.', 'testing'))

Preparing simulation
The simulation is running. Current progress: 99%data\balloon_track_markers.kml
The simulation is running. Current progress: 100%
Simulation completed.


In [6]:
kml_path = Path(os.path.join('data', 'balloon_track.kml'))
mark_path = Path(os.path.join('data', 'balloon_track_markers.kml'))
out_dir = Path('./results')
    
layers = kg.convert(kml_path, out_dir)
marklayers = kg.convert(mark_path, out_dir)

In [7]:
kw = {'location': [simEnvironment.launchSiteLat, simEnvironment.launchSiteLon], 'zoom_start': 8,}
track = os.path.join('results', 'balloon_track.geojson')
markers = os.path.join('results', 'balloon_track_markers.geojson')

m = folium.Map(tiles='OpenStreetMap', **kw)

style_function = lambda x: {'fillColor': '#00aaff'}

tracks = folium.GeoJson(track, style_function)



tracks.add_to(m)

#marks = folium.GeoJson(markers)

#marks.add_to(m)

print()



In [8]:
with open(markers) as f:
    landings = json.load(f)

import pandas as pd

impact = []
lon = []
lat = []
lon_sum = 0
lat_sum = 0

for feature in landings['features']:
    if feature['properties']['name'].find('landing') > 0:
        coords = feature['geometry']['coordinates']
        #print(coords)
        impact.append([coords[1],coords[0]])
        lon.append(coords[1])
        lat.append(coords[0])
        #llon.append(spot[1])
        #print(coords)
        lon_sum += coords[1]
        lat_sum += coords[0]
        
lonerr = statistics.stdev(lon)
laterr = statistics.stdev(lat)

#print(laterr, lon)

loncent = lon_sum / len(impact)
latcent = lat_sum / len(impact)

#print(impact)
#print(loncent, latcent)
#print(impact)
   
folium.CircleMarker([loncent,latcent], radius=10, color='black').add_to(m)

In [9]:
latp = []
lonp = []

latp.append([loncent-lonerr,latcent])
latp.append([loncent+lonerr,latcent])
lonp.append([loncent,latcent-laterr])
lonp.append([loncent,latcent+laterr])

folium.PolyLine(latp, color="green", weight=2.5, opacity=1).add_to(m)
folium.PolyLine(lonp, color="green", weight=2.5, opacity=1).add_to(m)

In [10]:
m

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [11]:
h = folium.Map(tiles='OpenStreetMap', location=[loncent,latcent], zoom_start=8,)

HeatMap(impact).add_to(h)
folium.Marker([loncent,latcent]).add_to(h)

h

In [12]:
h.save(os.path.join('results', 'map.html'))
